In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore') #para que no salgan mensajes que no nos interesan

df = pd.read_csv('synergy.csv')
print("Número de registros: ",len(df))
df.head()

Número de registros:  19056


,register_id,direction,origin,destination,year,date,product,transport_mode,company_name,total_value
0,1,Exports,Japan,China,2015,31/01/15,Cars,Sea,Honda,33000000
1,2,Exports,Japan,China,2015,01/02/15,Cars,Sea,Honda,16000000
2,3,Exports,Japan,China,2015,02/02/15,Cars,Sea,Honda,29000000
3,4,Exports,Japan,China,2015,03/02/15,Cars,Sea,Honda,14000000
4,5,Exports,Japan,China,2015,04/02/15,Cars,Sea,Honda,17000000


In [2]:
df_rutas = df.loc[:,['direction','origin','destination','total_value']]
df_rutas

,direction,origin,destination,total_value
0,Exports,Japan,China,33000000
1,Exports,Japan,China,16000000
2,Exports,Japan,China,29000000
3,Exports,Japan,China,14000000
4,Exports,Japan,China,17000000
...,...,...,...,...
19051,Imports,Japan,Singapore,1000000
19052,Imports,Malaysia,Singapore,2000000
19053,Imports,Malaysia,Singapore,33000000
19054,Imports,Malaysia,Singapore,13000000


# Obtener rutas sin repetición

In [3]:
rutas = df_rutas.loc[:,['origin','destination']].values
rutas = { tuple(row) for row in rutas} #quitar rutas repetidas, utilizando conjunto
rutas = list(rutas) #Volver el conjunto una lista
print(len(rutas))
rutas

172


[('Singapore', 'Malaysia'),
 ('Brazil', 'Argentina'),
 ('Russia', 'Germany'),
 ('Mexico', 'Singapore'),
 ('USA', 'United Arab Emirates'),
 ('South Korea', 'USA'),
 ('USA', 'Brazil'),
 ('Spain', 'Belgium'),
 ('Italy', 'Mexico'),
 ('Russia', 'Belorussia'),
 ('Australia', 'Mexico'),
 ('Switzerland', 'Russia'),
 ('South Korea', 'United Arab Emirates'),
 ('Japan', 'Spain'),
 ('Spain', 'Russia'),
 ('Russia', 'India'),
 ('South Korea', 'Brazil'),
 ('Japan', 'China'),
 ('Germany', 'Italy'),
 ('China', 'Belgium'),
 ('USA', 'Argentina'),
 ('Brazil', 'USA'),
 ('Russia', 'France'),
 ('Japan', 'Thailand'),
 ('Japan', 'Switzerland'),
 ('Netherlands', 'France'),
 ('Mexico', 'USA'),
 ('Belgium', 'Germany'),
 ('Germany', 'France'),
 ('Spain', 'Brazil'),
 ('Netherlands', 'Brazil'),
 ('China', 'Mexico'),
 ('France', 'Spain'),
 ('Italy', 'Singapore'),
 ('France', 'China'),
 ('Mexico', 'Brazil'),
 ('China', 'United Arab Emirates'),
 ('Japan', 'India'),
 ('Australia', 'Singapore'),
 ('France', 'United Kingd

# Para exportaciones

In [4]:
df_exp = df_rutas[df_rutas['direction']=='Exports']
print("Exportaciones totales: ",len(df_exp))
print("Valor total de exportaciones: ",df_exp['total_value'].sum())
df_exp.head()

Exportaciones totales:  15408
Valor total de exportaciones:  160163298000


,direction,origin,destination,total_value
0,Exports,Japan,China,33000000
1,Exports,Japan,China,16000000
2,Exports,Japan,China,29000000
3,Exports,Japan,China,14000000
4,Exports,Japan,China,17000000


# Función para contar el uso de la ruta

In [5]:
def contar(df):
    rutas_conteo = []
    for ruta in rutas:
        origen = ruta[0]
        destino = ruta[1]
        df_sum = df[(df['origin']==origen) & (df['destination']==destino)]
        conteo = len(df_sum)
        ruta = list(ruta) #volvemos las tuplas listas para agregarles el conteo con append  
        valor = np.sum(df_sum['total_value'])
        ruta.append(conteo) #No afecta a la tupla original asi que metemos estos datos en nueva lista
        ruta.append(valor)#Agregar valor total sumado de cada ruta
        rutas_conteo.append(ruta)
    #Contar cada ruta, se filtran con mascaras booleanas
    rutas_conteo.sort(reverse = True,key = lambda x:x[2])#Ordenarlas por mayor uso de ruta
    df_conteo = pd.DataFrame(rutas_conteo)#lo volvemos DataFrame
    return df_conteo   

## Función para presentar el top 10 y agregar porcentajes 
## sobre el total de exportaciones o importaciones

In [6]:
def val_10(df_completo):
    df = df_completo.iloc[0:10,:]
    df = df.copy()
    suma = (np.sum(df.iloc[:,2])/np.sum(df_completo.iloc[:,2]))*100
    suma = int(suma)
    suma = str(suma)+"%"
    valor = (np.sum(df.iloc[:,3])/np.sum(df_completo.iloc[:,3]))*100
    valor = int(valor)
    valor = str(valor)+'%'
    df.loc['Porcentaje'] = [0,0,suma,valor] 
    return df

## 10 Rutas más transitadas en exportación

In [7]:
df_conteo_exp = contar(df_exp) #llamado a la función
df_conteo_exp.drop(df_conteo_exp[df_conteo_exp[2]==0].index,inplace = True) #ELiminar si no hay exportaciones

df_10_exp = val_10(df_conteo_exp)

print("Por orden de mayor transito en la ruta")
#print('Total filas:',len(df_conteo_exp))
df_10_exp

Por orden de mayor transito en la ruta


,0,1,2,3
0,South Korea,Vietnam,497,6877007000
1,Netherlands,Belgium,437,3238142000
2,USA,Netherlands,436,1032187000
3,China,Mexico,330,12250000000
4,Japan,Brazil,306,3368155000
5,Germany,France,299,2902214000
6,South Korea,Japan,279,4594000000
7,Australia,Singapore,273,493000000
8,Canada,Mexico,261,8450000000
9,China,Spain,250,1862000000


## 10 Rutas con mayor valor en exportaciones

In [8]:
df_conteo_exp.sort_values([3],ascending = False,inplace=True)
print("Por orden de mayor valor")
df_10_exp_val = val_10(df_conteo_exp)
df_10_exp_val

Por orden de mayor valor


,0,1,2,3
3,China,Mexico,330,12250000000
8,Canada,Mexico,261,8450000000
0,South Korea,Vietnam,497,6877007000
14,France,Belgium,223,5538069000
28,France,United Kingdom,147,5427000000
23,China,South Korea,189,4790000000
22,USA,Mexico,194,4710000000
6,South Korea,Japan,279,4594000000
38,Germany,Italy,130,4541000000
33,China,Germany,142,4090000000


## Los datos concuerdan

In [9]:
print("Exportaciones totales: ",len(df_exp))
print("Suma de exportaciones: ",np.sum(df_conteo_exp.iloc[:,2]))
#Suma todas las rutas y coincide con el numero total de filas de df_conteo

Exportaciones totales:  15408
Suma de exportaciones:  15408


In [10]:
print("Valor total: ",df_exp['total_value'].sum())
print("Suma de valores: ",np.sum(df_conteo_exp.iloc[:,3]))

Valor total:  160163298000
Suma de valores:  160163298000


# Para importes

In [11]:
df_imp = df_rutas[df_rutas['direction']=='Imports']
print("Importaciones totales: ",len(df_imp))
print("Valor total de importación: ",df_imp['total_value'].sum())
df_imp.head()

Importaciones totales:  3648
Valor total de importación:  55528000000


,direction,origin,destination,total_value
15408,Imports,Mexico,USA,3000000
15409,Imports,Mexico,USA,13000000
15410,Imports,Mexico,USA,40000000
15411,Imports,Mexico,USA,48000000
15412,Imports,Mexico,USA,44000000


# Ordenados por mayor transito de la ruta

In [12]:
df_conteo_imp = contar(df_imp)
df_conteo_imp.drop(df_conteo_imp[df_conteo_imp[2]==0].index,inplace = True)
print("Por orden de mayor transito en la ruta")

df_10_import = val_10(df_conteo_imp)
df_10_import

Por orden de mayor transito en la ruta


,0,1,2,3
0,Singapore,Thailand,273,4017000000
1,Germany,China,233,1328000000
2,China,Japan,210,3237000000
3,Japan,Mexico,206,3918000000
4,China,Thailand,200,3831000000
5,Malaysia,Thailand,195,3482000000
6,Spain,Germany,142,2044000000
7,Mexico,USA,122,2365000000
8,China,United Arab Emirates,114,3357000000
9,Brazil,China,113,413000000


# Ordenados por mayor valor

In [13]:
df_conteo_imp.sort_values([3],ascending = False,inplace=True)
print("Por orden de mayor valor")

df_10_import_val = val_10(df_conteo_imp)
df_10_import_val

Por orden de mayor valor


,0,1,2,3
0,Singapore,Thailand,273,4017000000
3,Japan,Mexico,206,3918000000
4,China,Thailand,200,3831000000
5,Malaysia,Thailand,195,3482000000
8,China,United Arab Emirates,114,3357000000
2,China,Japan,210,3237000000
7,Mexico,USA,122,2365000000
16,Japan,United Arab Emirates,76,2238000000
6,Spain,Germany,142,2044000000
18,Germany,Mexico,70,1804000000


## Los datos concuerdan

In [14]:
print("Importaciones totales: ",len(df_imp))
print("Suma de importaciones: ",np.sum(df_conteo_imp.iloc[:,2]))

Importaciones totales:  3648
Suma de importaciones:  3648


In [15]:
print("Valor total: ",df_imp['total_value'].sum())
print("Suma de valores: ",np.sum(df_conteo_imp.iloc[:,3]))

Valor total:  55528000000
Suma de valores:  55528000000


# Transporte

In [16]:
dft = df.copy()

destination = ['Exports','Imports']
transport_mode = dft['transport_mode'].unique()

Exports = []
Imports = []

dft = dft.set_index(['direction','transport_mode'])
for destino in destination:
    for transporte in transport_mode:
        dft2 = dft.loc[destino,transporte]
        if destino=='Exports':
            Exports.append([transporte,len(dft2),dft2['total_value'].sum()])
        else:
            Imports.append([transporte,len(dft2),dft2['total_value'].sum()])
            
Exports.sort(reverse = True,key = lambda x:x[2])
Imports.sort(reverse = True,key = lambda x:x[2])

print('Exportaciones',Exports)
print('Importaciones',Imports)

Exportaciones [['Sea', 8434, 65592622000], ['Rail', 2780, 34505043000], ['Air', 2045, 32785147000], ['Road', 2149, 27280486000]]
Importaciones [['Sea', 2254, 34938000000], ['Rail', 601, 9123000000], ['Road', 449, 5990000000], ['Air', 344, 5477000000]]


# Mezcla opción 2 y 3

## Dejamos el top 3 medios de transporte, luego los paises que generan el 80% cuando se tienen todas las exportaciones

In [17]:
df4 = df.set_index(['direction','transport_mode'])
df_exports = df4.loc['Exports']
print('Total exportaciones: ',len(df_exports))

df_exports = df_exports.loc[['Sea','Rail','Air']]
df_exports = df_exports.set_index('origin')
df_exports = df_exports.loc[['China', 'France', 'USA', 'South Korea', 'Russia', 'Japan', 'Germany', 'Canada']]

valor = df_exports['total_value'].sum()

print("porcentaje de rutas: ",((8018/15408)*100))
print("porcentaje de valor: ",((valor/160163298000)*100))
display(df_exports)

Total exportaciones:  15408
porcentaje de rutas:  52.03790238836968
porcentaje de valor:  70.72390267588021


,register_id,destination,year,date,product,company_name,total_value
origin,,,,,,,
China,581,Japan,2018,07/06/18,Smartphones,Xiaomi Corp,21000000
China,582,Japan,2018,08/06/18,Smartphones,Xiaomi Corp,47000000
China,583,Japan,2018,09/06/18,Smartphones,Xiaomi Corp,22000000
China,584,Japan,2018,10/06/18,Smartphones,Xiaomi Corp,50000000
China,585,Japan,2018,11/06/18,Smartphones,Xiaomi Corp,49000000
...,...,...,...,...,...,...,...
Canada,6800,Mexico,2017,28/05/17,Gold,Kinross Gold‎ Co,5000000
Canada,6801,Mexico,2017,29/05/17,Gold,Kinross Gold‎ Co,14000000
Canada,6802,Mexico,2017,30/05/17,Wood,Black Forest Wood Co,6000000


In [18]:
df5 = df.set_index(['direction','transport_mode'])
df_imports = df5.loc['Imports']
print('Total importaciones: ',len(df_imports))
df_imports = df_imports.loc[['Sea','Rail','Road']]
df_imports = df_imports.set_index('origin')
df_imports = df_imports.loc[['China', 'Japan', 'USA', 'Mexico', 'Germany', 'Singapore', 'South Korea', 'Malaysia']]

valor_2 = df_imports['total_value'].sum()

print("porcentaje de rutas: ",((2460/3648)*100))
print("porcentaje de valor: ",((valor_2/55528000000)*100))
display(df_imports)


Total importaciones:  3648
porcentaje de rutas:  67.43421052631578
porcentaje de valor:  72.89655669211929


,register_id,destination,year,date,product,company_name,total_value
origin,,,,,,,
China,15501,USA,2015,03/05/15,Machinery and electronics,Lenovo Group Ltd,5000000
China,15502,USA,2015,04/05/15,Machinery and electronics,Lenovo Group Ltd,34000000
China,15503,USA,2015,05/05/15,Machinery and electronics,Lenovo Group Ltd,30000000
China,15504,USA,2015,06/05/15,Machinery and electronics,Lenovo Group Ltd,32000000
China,15505,USA,2015,07/05/15,Machinery and electronics,Lenovo Group Ltd,38000000
...,...,...,...,...,...,...,...
Malaysia,18839,Thailand,2019,29/04/19,Palm oil,Green Oil Thailand Co,21000000
Malaysia,18840,Thailand,2019,30/04/19,Palm oil,Green Oil Thailand Co,15000000
Malaysia,18841,Thailand,2019,01/05/19,Palm oil,Green Oil Thailand Co,20000000


## No aplicamos el filtro de top 3 transportes para importación, y solo tomamos los paises que dan 80% de valor

In [24]:
df6 = df.set_index(['direction','transport_mode'])
df_imports = df6.loc['Imports']
print('Total importaciones: ',len(df_imports))
df_imports = df_imports.set_index('origin')
df_imports = df_imports.loc[['China', 'Japan', 'USA', 'Mexico', 'Germany', 'Singapore', 'South Korea', 'Malaysia']]

valor_2 = df_imports['total_value'].sum()

print("porcentaje de rutas: ",((2804/3648)*100))
print("porcentaje de valor: ",((valor_2/55528000000)*100))
display(df_imports)


Total importaciones:  3648
porcentaje de rutas:  76.8640350877193
porcentaje de valor:  82.7600489842962


,register_id,destination,year,date,product,company_name,total_value
origin,,,,,,,
China,15501,USA,2015,03/05/15,Machinery and electronics,Lenovo Group Ltd,5000000
China,15502,USA,2015,04/05/15,Machinery and electronics,Lenovo Group Ltd,34000000
China,15503,USA,2015,05/05/15,Machinery and electronics,Lenovo Group Ltd,30000000
China,15504,USA,2015,06/05/15,Machinery and electronics,Lenovo Group Ltd,32000000
China,15505,USA,2015,07/05/15,Machinery and electronics,Lenovo Group Ltd,38000000
...,...,...,...,...,...,...,...
Malaysia,18843,Thailand,2019,03/05/19,Palm oil,Green Oil Thailand Co,12000000
Malaysia,19053,Singapore,2020,28/06/20,Gas turbines,Union Energy Co,2000000
Malaysia,19054,Singapore,2020,29/06/20,Gas turbines,Union Energy Co,33000000


## Notas

In [19]:
{(1,2),(1,3)} #Solo se pueden hacer conjuntos de tuplas

{(1, 2), (1, 3)}

In [20]:
df_exp[(df_exp['origin']=='China') & (df_exp['destination']=='Mexico')]

,direction,origin,destination,total_value
618,Exports,China,Mexico,45000000
619,Exports,China,Mexico,74000000
620,Exports,China,Mexico,21000000
621,Exports,China,Mexico,54000000
622,Exports,China,Mexico,21000000
...,...,...,...,...
12406,Exports,China,Mexico,4000000
12407,Exports,China,Mexico,11000000
12408,Exports,China,Mexico,0
12409,Exports,China,Mexico,4000000


In [21]:
#Para sumar valores de una columna especifica, despues de filtrar
df_sum_2 = df_exp[(df_exp['origin']=='Japan') & (df_exp['destination']=='Mexico')]
np.sum(df_sum_2['total_value'])

1911000000

In [22]:
origen = df['origin'].unique()
destino = df['destination'].unique()

display(origen)
display(destino)

array(['Japan', 'Germany', 'China', 'Italy', 'USA', 'Russia',
       'South Korea', 'Netherlands', 'France', 'Canada', 'Belgium',
       'Spain', 'India', 'United Kingdom', 'Australia', 'Brazil',
       'Switzerland', 'Mexico', 'Austria', 'Singapore', 'Vietnam',
       'Malaysia', 'United Arab Emirates'], dtype=object)

array(['China', 'USA', 'South Korea', 'Spain', 'Italy', 'Japan',
       'Germany', 'Mexico', 'Brazil', 'France', 'Switzerland',
       'United Kingdom', 'Croatia', 'Ireland', 'Belgium', 'Argentina',
       'Singapore', 'Canada', 'Netherlands', 'Belorussia', 'Turkey',
       'India', 'Vietnam', 'Russia', 'United Arab Emirates', 'Thailand',
       'Philippines', 'Peru', 'Austria', 'New Zealand', 'Rusia',
       'Slovakia', 'Malaysia', 'Poland'], dtype=object)